## Etapa 9 : Analisando os Rótulos gerados

Nesta etapa, vamos analisar as diferenças de rotulagem e definir o rótulo que será usado para o modelo final

In [1]:
# importa funções padroes
import pandas as pd 
import numpy as np
import re

In [2]:
#importa dataset
tweets = pd.read_csv('dataset_com_sentimento.csv', index_col=0)
tweets

,in_reply_to_screen_name,full_text,sentimento,sentimento_CNN,sentimento_RNN,pesos_RNN
0,santander_br,isso qualquer operacao que eu tente fazer em ...,0,0,0,0.002110
1,santander_br,a atendente pagou pela segunda vez a fatura d...,0,1,0,0.199904
2,Bancointer,pode trazer o meus,1,1,1,0.998421
3,santander_br,nao estou conseguindo gerar cartao compra onl...,0,0,0,0.001986
4,itau,mas foi e o setup foi bem facil muito bom,1,1,1,0.997619
...,...,...,...,...,...,...
132370,amazonBR,tambem naoficosem livros,1,1,1,0.592652
132371,amazonBR,voces me confirmam se esse e mail foi enviado...,0,0,0,0.002341
132372,santander_br,me obrigou a atualizar o ap e agora so serve ...,0,0,0,0.050323
132373,Bancointer,decepcao nao ter opcao para pagar darf sem co...,0,0,0,0.001286


In [15]:
tweets[(tweets["sentimento"] != tweets["sentimento_CNN"]) | (tweets["sentimento"] != tweets["sentimento_RNN"])]

,in_reply_to_screen_name,full_text,sentimento,sentimento_CNN,sentimento_RNN,pesos_RNN,sentimento_medio
1,santander_br,a atendente pagou pela segunda vez a fatura d...,0,1,0,0.199904,0
7,lojasamericanas,esta dizendo que nao posso enviar mensagem pa...,0,1,0,0.041578,0
11,santander_br,moro no interior nao tem caixa eletronico,0,1,0,0.475362,0
14,C6Bank,sem burocracia abra sua conta gratuitamente di...,0,0,1,0.887948,0
16,C6Bank,contactless nao precisa inserir o cartao na ma...,1,1,0,0.060138,1
...,...,...,...,...,...,...,...
132361,amazonBR,ate quando essa promocao,0,0,1,0.556049,0
132363,Bancointer,foi efetuada dia,0,1,1,0.992992,1
132365,Bancointer,e necessario possuir patrimonio para abrir a ...,0,1,0,0.001743,0
132369,amazonBR,como ganha um desses,0,0,1,0.994699,0


Aproximada 30% dos tweets tiveram alguma divergencia entre os 3 modelos.

In [17]:
len(tweets[(tweets["sentimento"] != tweets["sentimento_CNN"])])

28343

In [18]:
len(tweets[(tweets["sentimento"] != tweets["sentimento_RNN"])])

28406

In [19]:
len(tweets[(tweets["sentimento_RNN"] != tweets["sentimento_CNN"])])

19605

Em aproximadamente 23% dos tweets existe divergencia entre a predição da SVM contra a predição de algum modelo de deep learning, ja a diferença entre os modelos de deep learning cai para 15%. 

In [24]:
len(tweets[(tweets["sentimento"] != tweets["sentimento_CNN"]) & ((tweets["pesos_RNN"] > 0.9) | (tweets["pesos_RNN"] < 0.1))])

18502

In [25]:
len(tweets[(tweets["sentimento"] != tweets["sentimento_RNN"]) & ((tweets["pesos_RNN"] > 0.9) | (tweets["pesos_RNN"] < 0.1))])

17560

In [23]:
len(tweets[(tweets["sentimento_RNN"] != tweets["sentimento_CNN"]) & ((tweets["pesos_RNN"] > 0.9) | (tweets["pesos_RNN"] < 0.1))])

10682

Este valor cai quase pela metade se avaliarmos apenas os tweets com um grau alto de certeza.

Para treinar nosso modelo final, vamos utilizar a mediana dos rotulos dos 3 modelos.

In [4]:
colunas = tweets.loc[: , "sentimento":"sentimento_RNN"]

In [8]:
media = []
for i in colunas.mean(axis=1):
    if i > 0.5:
        media.append(1)
    else:
        media.append(0)

In [10]:
tweets['sentimento_medio'] = media

In [12]:
tweets.head(20)

,in_reply_to_screen_name,full_text,sentimento,sentimento_CNN,sentimento_RNN,pesos_RNN,sentimento_medio
0,santander_br,isso qualquer operacao que eu tente fazer em ...,0,0,0,0.002110,0
1,santander_br,a atendente pagou pela segunda vez a fatura d...,0,1,0,0.199904,0
2,Bancointer,pode trazer o meus,1,1,1,0.998421,1
3,santander_br,nao estou conseguindo gerar cartao compra onl...,0,0,0,0.001986,0
4,itau,mas foi e o setup foi bem facil muito bom,1,1,1,0.997619,1
5,lojasamericanas,cara e inadmissivel o que minha esposa passou...,0,0,0,0.000957,0
6,lojasamericanas,poderia me ajudar nao consigo encontrar os te...,0,0,0,0.323144,0
7,lojasamericanas,esta dizendo que nao posso enviar mensagem pa...,0,1,0,0.041578,0
8,lojasamericanas,mas eu estou seguindo,1,1,1,0.997538,1
9,santander_br,ola gente bom dia quero entra na minha conta ...,1,1,1,0.831016,1


In [26]:
tweets.to_csv('dataset_com_sentimento.csv')

print('Arquivo Gravado com Sucesso')

Arquivo Gravado com Sucesso
